# Session 6 exercises

These are sample answers for the in-class exercises in Session 6 of PHAS0030.  You should make sure that you can do these yourself! The further work exercises will be in a separate notebook.

In [ ]:
# We always start with appropriate imports; note the use of the IPython magic
# command to set up Matplotlib within the notebook
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## 3. The classical wave equation

### Question 1

In [ ]:
def explicit_wave_eq_update(theta_n, theta_nm1,r):
    """Update wave equation using simple finite difference 
    approach.  Assumes periodic boundary conditions.
    Inputs: 
    theta_n   Wave at time t_n     = n*dt
    theta_nm1 Wave at time t_{n-1} = (n-1)*dt
    r         Constant (c dt/dx)
    Output:
    theta at time t_{n+1} = (n+1)*dt """
    theta_np1 = 2.0*(1-r*r)*theta_n - theta_nm1 + r*r*(np.roll(theta_n,1) + np.roll(theta_n,-1))
    return theta_np1

### Question 2(a)

In [ ]:
wavelength = 1                  #m
wavevector = 2*np.pi/wavelength # 1/m
frequency = 1                   # Hz
ang_freq = 2*np.pi*frequency    # 1/s
speed = wavelength*frequency    # m/s

# Define number of points in a wavelength and factor r
N = 20
r = 0.1
# Initialise
dx = wavelength/N
dt = r*dx/speed
# Set up physical domain of wavefunction, remembering that
# np.arange excludes the final point (important!)
q1_x = np.arange(0,3*wavelength,dx)

In [ ]:
print(f"dt is {dt:.5f}s")
theta_next = np.zeros_like(q1_x)
# Initial conditions: wave at t=0...
t = 0
theta_0 = np.sin(wavevector*q1_x - ang_freq*t)
# ...and t=dt
t += dt
theta_1 = np.sin(wavevector*q1_x - ang_freq*t)
# Now step forward 200 steps without storing wave
iters = 200
for n in range(2,iters):
    t += dt
    # New theta
    theta_next = explicit_wave_eq_update(theta_1,theta_0,r)
    # Cycle storage
    theta_0 = theta_1
    theta_1 = theta_next

### Question 2(b)

Now plot the wave and the exact solution; below, we plot the difference to find out how large it is.

In [ ]:
plt.plot(q1_x,np.sin(wavevector*q1_x - ang_freq*dt*(iters-1)),label='Exact')
plt.plot(q1_x,theta_next,label='Numerical')
plt.legend()
plt.xlabel('x in m')
plt.ylabel(r'$\theta$')
plt.title('Wave propagation with r=0.1')

In [ ]:
plt.plot(q1_x,np.sin(wavevector*q1_x - ang_freq*dt*(iters-1))-theta_next)
plt.xlabel('x in m')
plt.ylabel(r'$\theta_{exact}$ - $\theta_{num}$')
plt.title('Difference between exact and numerical')

These compare extremely well.  If we plot the difference (as above), it is around 2%.

### Question 2(c)

Now I show an example where we save the wave at all time points, and plot the resulting 2D array using `plt.imshow`.

In [ ]:
iters = 200
# Set up storage for all time steps
theta_store = np.zeros((iters+2,len(q1_x)))
# Initial conditions: wave at t=0...
t = 0
theta_store[0] = np.sin(wavevector*q1_x - ang_freq*t)
# ...and t=dt
t += dt
theta_store[1] = np.sin(wavevector*q1_x - ang_freq*t)
# Now step forward 200 steps, storing wave at each step
for n in range(2,iters+2):
    t += dt
    theta_store[n] = explicit_wave_eq_update(theta_store[n-1],theta_store[n-2],r)
print(theta_store)
# Plot using imshow
plt.imshow(theta_store,origin='lower',extent=(0,3*wavelength,0,dt*iters))
plt.colorbar(label='Amplitude(m)')
plt.xlabel('x in m')
plt.ylabel('t in s')
plt.title('Wave propagation with r=0.1')

The gradient of the stripes clearly gives the phase velocity; this gives a nice overview of the general behaviour of the wave, but quantitative information might be easier with ordinary line plots.

### Question 2(d)


In [ ]:
# Storage for Nr values of r
Nr = 6
store = np.zeros((Nr,len(q1_x)))
endt = np.zeros(Nr)
rvec = np.linspace(0.05,0.55,Nr)
# Iterate over values of r
for i, r in enumerate(rvec):
    # Initialise
    dt = r*dx/speed
    t = 0
    theta_0 = np.sin(wavevector*q1_x - ang_freq*t)
    t += dt
    theta_1 = np.sin(wavevector*q1_x - ang_freq*t)
    iters = 203
    for n in range(2,iters):
        theta_next = explicit_wave_eq_update(theta_1,theta_0,r)
        theta_0 = theta_1
        theta_1 = theta_next
    # Store final waveform and time for comparison
    store[i] = np.copy(theta_next)
    endt[i] = (iters-1)*dt
for i in range(Nr):
    dt = rvec[i]*dx/speed
    # This line below calculates the numerical angular frequency: 
    # it is not actually the same as the actual angular frequency
    # However this is a (very!) subtle point which goes beyond the
    # course, so we don't describe it, and comment this out by default
    # omega_num =  (2/dt)*np.arcsin(rvec[i]*np.sin(wavevector*0.5*dx))
    omega_num = ang_freq 
    print(f"Exact: {ang_freq}, numerical: {omega_num}")
    plt.plot(q1_x,store[i]-np.sin(wavevector*q1_x - omega_num*endt[i]),label=f'r={rvec[i]:.3f}')
plt.legend()
plt.xlabel('x in m')
plt.ylabel(r'Error in $\theta$')
plt.title('Effect of varying r')

This appears to be reasonable behaviour: as we increase the value of `r`, the accuracy decreases (though the computational effort required to reach a certain time will also decrease, because $dt$ will increase).  However, it is quite easy to find anomalous behaviour (e.g. I found that going from 190 to 200 to 210 iterations changes which method is accurate and which is not).

What is happening here is that the phase velocity of the numerical solution is not quite equal to the phase velocity of the analytic solution, and will vary with `r`, so for large numbers of iterations we accumulate a phase shift between the numerical and analytic solutions; as we change the number of iterations, the phase shift for each value of `r` changes, moving the different solutions into and out of phase.

The difference between the numerical and analytic phase velocities is reduced by increasing the number of points in a wave (i.e. reducing `dx`).  So you could increase this (`N=50` or `N=100` above) or reduce the number of iterations to 20.  Doing this enables us to check the effect of `r`.  (These notes are for your information: I would not expect you to reproduce this in an exam!)

### Question 3

This time, I will use the figure-based method of plotting, and add a subplot at each iteration.

In [ ]:
r = 0.1
graph = 1
# The argument tight_layout spaces the plots nicely
fig_S3Q3 = plt.figure(tight_layout=True)
ax_S3Q3 = []
for N in (5,10,20,50):
    ax_S3Q3.append(fig_S3Q3.add_subplot(2,2,graph))
    # Initialise
    dx = wavelength/N
    q3_x = np.arange(0,3*wavelength,dx)
    dt = r*dx/speed
    t = 0
    theta_0 = np.sin(wavevector*q3_x - ang_freq*t)
    t += dt
    theta_1 = np.sin(wavevector*q3_x - ang_freq*t)
    iters = 200
    for n in range(iters):
        theta_next = explicit_wave_eq_update(theta_1,theta_0,r)
        theta_0 = theta_1
        theta_1 = theta_next
    ax_S3Q3[graph-1].plot(q3_x,theta_next)
    ax_S3Q3[graph-1].plot(q3_x,np.sin(wavevector*q3_x - ang_freq*dt*iters))
    ax_S3Q3[graph-1].set_title(f'N={N}')
    graph+=1
ax_S3Q3[0].set_ylabel("Wave amplitude in m")
ax_S3Q3[2].set_ylabel("Wave amplitude in m")
ax_S3Q3[2].set_xlabel("Distance in m")
ax_S3Q3[3].set_xlabel("Distance in m")
fig_S3Q3.suptitle("Comparing accuracy for varying N")

Here we see that too small a number of points in $x$ is badly wrong, but even with a small number we do quite well.  For a quantitative discussion we would want to plot the *difference* between the analytic and numerical solutions.

## 4. Two dimensions

### Question 1

Note that I've done the $x$ and $y$ derivatives separately to make it clearer, but this could easily be combined into `4.0*theta_n` for efficiency.

In [ ]:
def explicit_2D_wave_eq_update(theta_n, theta_nm1,r):
    """Update wave equation using simple finite difference 
    approach.  Assumes periodic boundary conditions.
    Inputs: 
    theta_n   Wave at time t_n     = n*dt
    theta_nm1 Wave at time t_{n-1} = (n-1)*dt
    r         Constant (c dt/dx)
    Output:
    theta at time t_{n+1} = (n+1)*dt """
    theta_np1 = 2.0*theta_n - theta_nm1 + r*r*(np.roll(theta_n,1,axis=0) - # x axis
                                               2.0*theta_n + np.roll(theta_n,-1,axis=0)
                                      ) + r*r*(np.roll(theta_n,1,axis=1) - # y axis
                                               2.0*theta_n + np.roll(theta_n,-1,axis=1))
    return theta_np1

### Question 2

In [ ]:
# Define number of points
N = 100
# Initialise
wavelength = 1 #m
wavevector = 2*np.pi/wavelength
frequency = 1 # Hz
ang_freq = 2*np.pi*frequency
speed = wavelength*frequency
dx = wavelength/N # Also dy
S4Q2_x = np.arange(0,3*wavelength,dx)
S4Q2_y = np.arange(0,3*wavelength,dx)
# Use np.meshgrid to make 2D arrays of x and y
S4_x2d, S4_y2d = np.meshgrid(S4Q2_x,S4Q2_y)

### Question 3

Note that it's important to create the initial wave using the 2D $x$ and $y$ arrays generated by `np.meshgrid`, otherwise the initial wave won't be 2D.

In [ ]:
midy = 1.5*wavelength
sigma = 1.0
# Start with a sine wave in x, with Gaussian envelope in y
t = 0
theta_0 = np.sin(wavevector*S4_x2d - ang_freq*t)*np.exp(-(S4_y2d-midy)**2/sigma)
# Now zero beyond 1m
theta_0[:,N:] = 0.0
theta_0[:,0] = 0.0
theta_0[0] = 0.0
theta_0[-1] = 0.0
plt.imshow(theta_0,extent=(0,3,0,3))
plt.colorbar()

Let's also use the 3D plotting facility; note that you can set the view point using `ax.view_init(elevation, azimuth)`.

In [ ]:
fig_3d1 = plt.figure(tight_layout=True)
ax3d1 = fig_3d1.add_subplot(111,projection='3d')
surf = ax3d1.plot_surface(S4_x2d,S4_y2d,theta_0,cmap='viridis')
ax3d1.set_xlabel('x')
ax3d1.set_ylabel('y')
ax3d1.set_zlabel(r'$\theta$')
# Notice how we can add a colorbar using the surface (what we want to colour) and 
# the axis (what we attach the colorbar to)
fig_3d1.colorbar(surf,ax=ax3d1)
ax3d1.view_init(50, 110)

### Question 4

In [ ]:
r = 0.2
dt = r*dx/speed
t += dt
theta_1 = np.sin(wavevector*S4_x2d - ang_freq*t)*np.exp(-(S4_y2d-midy)**2/sigma)
# Confine starting wave to left hand side
theta_0[:,N:] = 0.0
theta_1[:,N:] = 0.0
theta_1[:,0] = 0.0
# Boundary conditions at top and bottom
theta_0[0,:] = 0.0
theta_0[3*N-1,:] = 0.0
theta_1[0,:] = 0.0
theta_1[3*N-1,:] = 0.0
iters = 100
for n in range(iters):
    # Update
    theta_next = explicit_2D_wave_eq_update(theta_1,theta_0,r)
    # Boundary conditions: hard walls at top & bottom
    theta_next[0,:] = 0.0
    theta_next[3*N-1,:] = 0.0
    # Update
    theta_0 = theta_1
    theta_1 = theta_next

In [ ]:
plt.imshow(theta_next,extent=(0,3,0,3))
plt.colorbar()

Notice how there is some small artefact near the top and bottom; this must come from the boundary conditions and the initial wave not quite being zero near the edges.

### Question 5

In [ ]:
iters = 1200
figS4Q5 = plt.figure(figsize=(10,6),tight_layout=True)
# Start with a sine wave in x, with Gaussian envelope in y
t = 0
theta_0 = np.sin(wavevector*S4_x2d - ang_freq*t)*np.exp(-(S4_y2d-midy)**2/sigma)
# Now zero beyond 1m
theta_0[:,N:] = 0.0
theta_0[:,0] = 0.0
theta_0[0] = 0.0
theta_0[-1] = 0.0
t += dt
theta_1 = np.sin(wavevector*S4_x2d - ang_freq*t)*np.exp(-(S4_y2d-midy)**2/sigma)
# Confine starting wave to left hand side
theta_0[:,N:] = 0.0
theta_1[:,N:] = 0.0
theta_1[:,0] = 0.0
# Boundary conditions at top and bottom
theta_0[0,:] = 0.0
theta_0[3*N-1,:] = 0.0
theta_1[0,:] = 0.0
theta_1[3*N-1,:] = 0.0
index = 1
for n in range(iters):
    # Update
    theta_next = explicit_2D_wave_eq_update(theta_1,theta_0,r)
    # Boundary conditions: hard walls at top & bottom
    theta_next[0] = 0.0
    theta_next[-1] = 0.0
    # Update
    theta_0 = theta_1
    theta_1 = theta_next
    if n%100==0:
        ax = figS4Q5.add_subplot(3,4,index)
        image = ax.imshow(theta_next)
        ax.set_title(f'N={n}')
        figS4Q5.colorbar(image)
        index+=1

## 5. Time-dependent Schrodinger equation

### Question 1

Note that the value of `sigma` has a strong effect on the propagation; you might like to experiment if you have time.

In [ ]:
Nx = 401
x = np.linspace(-100,100,Nx)
k = 1
sigma = 10.0
x0 = -75.0
psi0 = np.exp(1j*k*x)*np.exp(-(x-x0)**2/sigma**2)

In [ ]:
plt.plot(x,psi0.real,label='real')
plt.plot(x,psi0.imag,label='imaginary')
plt.xlabel("Position in bohr radii")
plt.ylabel("Amplitude")
plt.title("Initial wavefunction")
plt.legend()

### Question 2

Create the matrices; notice how we add the array for V to the main diagonal.  The process for making the matrices is identical to the process we used in Session 5 for the heat/diffusion equation.  You could also make $\mathbf{N}$ as the complex conjugate of $\mathbf{M}$.  (It's always worth being very careful with complex matrices; these are symmetric, so we don't need to worry about Hermitian conjugates.)

I've used `np.full` to generate the 1D arrays (it returns an array of given shape, filled with the value specified - check the help if you want more information) but you could equally well do something like `A*np.ones(N,dtype=complex)` for some appropriate `A`.

In [ ]:
def calc_M(N,zeta,V,dt):
    """Calculate matrix M for Crank-Nicolson solution of TDSE
    Inputs: 
    N    size of matrix
    zeta parameter
    V    potential (array)
    dt   time step
    Outputs:
    (NxN) matrix"""
    maindiag = np.full(N,2.0*(2.0+1j*zeta)) + 2j*dt*V
    offdiag = np.full(N-1,-1j*zeta)
    output = np.diag(maindiag) + np.diag(offdiag,k=1) + np.diag(offdiag,k=-1)
    return output

In [ ]:
def calc_N(N,zeta,V,dt):
    """Calculate matrix N for Crank-Nicolson solution of TDSE
    Inputs: 
    N    size of matrix
    zeta parameter
    V    potential (array)
    dt   time step
    Outputs:
    (NxN) matrix"""
    maindiag = np.full(N,2.0*(2.0-1j*zeta)) - 2j*dt*V
    offdiag = np.full(N-1,1j*zeta)
    output = np.diag(maindiag) + np.diag(offdiag,k=1) + np.diag(offdiag,k=-1)
    return output

### Question 3

In [ ]:
V = np.zeros_like(x,dtype=complex)
dx = 0.5
dt = 0.1
zeta = dt/(dx*dx)
matM = calc_M(Nx,zeta,V,dt)
matN = calc_N(Nx,zeta,V,dt)
# Print matrices if you want to check
#print(matM)
#print(matN)
matMinv = np.linalg.inv(matM)
# Create boundary condition vector (the boundaries are set to zero)
b = np.zeros(Nx,dtype=complex)
matMinvN = np.dot(matMinv,matN)
matMinvb = np.dot(matMinv,b) # Not strictly necessary as b is zero!


### Question 4

In [ ]:
iters = 1200
psi0[0] = 0.0
psi0[-1] = 0.0
psi_this = np.copy(psi0)
figS5Q4 = plt.figure(figsize=(10,6),tight_layout=True)
index = 1
for i in range(iters):
    psi_next = matMinvb + np.dot(matMinvN,psi_this)
    psi_this = psi_next
    if i%100 == 0:
        ax = figS5Q4.add_subplot(3,4,index)
        ax.plot(x,psi_this.real)
        ax.set_title(f"i={i}")
        ax.set_ylim(-1,1) # Ensure consistent plot range for clarity
        ax.set_xlabel('x')
        ax.set_ylabel(r'$\psi$')
        index += 1

Notice how the wavepacket spreads as it propagates: it is not an eigenfunction of the Hamiltonian, and so different frequencies will travel at different speeds, leading to a change in the shape of the wavepacket.

## 7. Time-independent Schrodinger equation

### Question 1

This is straightforward: we're just stepping along $x$ and using the formula:

$$\psi_{i} = 2\psi_{i-1} + 2\Delta x^{2}(V_{i-1} - E)\psi_{i-1} - \psi_{i-2}$$

Note how the code is a direct implementation of this formula.

In [ ]:
def simple_FD_update(psi0,psi1,V,E,dx,Nx):
    """Perform simple integration for TISE based on 
    second-order FD expansion of differential.
    
    Inputs:
    psi0   Value of wavefunction at first point
    psi1   Value of wavefunction at second point
    V      Array of potential values
    E      Energy
    dx     Spacing in x
    Nx     Number of points in x (inc 0 and 1)
    
    Output:
    psi         Array of values of wavefunction
    """
    psi = np.zeros(Nx,dtype=complex)
    psi[0] = psi0
    psi[1] = psi1
    for i in range(2,Nx):
        psi[i] = 2.0*psi[i-1] + 2*dx*dx*(V[i-1]-E)*psi[i-1] - psi[i-2]
    return psi

### Question 2

In [ ]:
dx = 0.01
xmin = -5
xmax = 5
width = xmax - xmin
# Set number of points so that we include both start and end
Nx = int((xmax-xmin)/dx)+1
x = np.linspace(xmin,xmax,Nx)
# This is a zero potential (an infinite square well)
V = np.zeros_like(x,dtype=complex)
psi = np.zeros_like(x,dtype=complex)
# psi0 is required for boundary conditions; psi1 is arbitrary
psi0 = 0.0 + 0.0j
psi1 = 0.1 + 0.0j

### Question 3

We note that we have been lucky (or informed!) in choosing guesses for the energy that clearly bracket a solution.

In [ ]:
E0 = 0.0
E1 = 0.1
out0 = simple_FD_update(psi0,psi1,V,E0,dx,Nx)
out1 = simple_FD_update(psi0,psi1,V,E1,dx,Nx)
plt.plot(x,out0.real,label=f'E={E0}')
plt.plot(x,out1.real,label=f'E={E1}')
plt.xlabel('x')
plt.ylabel(r'$\psi$')
plt.title('Trial wavefunctions')
plt.legend()

### Question 4

We're building on our existing brackets.  Note that $E$ is the independent variable, and the value of the wavefunction at the end of the box is the function we're trying to get to zero.

In [ ]:
# E0 and E1 defined above; here are the corresponding function values
f0 = out0[-1]
f1 = out1[-1]

# I've used secant here; bisection is also fine (and a little easier to code)
n = 0
tol = 1e-4
while abs(E1 - E0) > tol:
    n += 1
    # Calculate next point
    dE = E1 - E0
    df = f1 - f0
    Enext = E1 - f1 * dE / df
    # Update storage
    E0 = E1
    E1 = Enext
    f0 = f1
    f1 = simple_FD_update(psi0,psi1,V,E1,dx,Nx)[-1]
# NB E0 becomes complex because it is calculated using f0 and f1
print(f"After {n} iterations, energy is {E0.real:.6e} Ha")
print(f"Difference to exact energy is {E0.real-0.5*(np.pi/10)**2:.6e} Ha")
plt.plot(x,simple_FD_update(psi0,psi1,V,E0,dx,Nx).real)
plt.xlabel("x in Bohr radii")
plt.ylabel("Amplitude")
plt.title("Ground state wavefunction (unnormalised)")

Note that we have not normalised the wavefunction; this is important in any practical applications, but is easy to do.